# Calculate optimal routes for six trucks with multiple waypoints


**Table of contents**:
* [Introduction](#Introduction)
* [Calculate optimized truck route (returning to the starting point)](#Calculate-optimized-truck-route-(returning-to-the-starting-point))
* [Calculate all routes](#Calculate-all-routes)
* [Display calculated routes on a Map](#Display-calculated-routes-on-a-Map)

## Introduction

When we have a fleet of trucks that have to distribute stock to several shops, we will face the **challenge of calculating the optimal route for each truck**.  
![Animation of truck optimized routes](./images/truck_optimized_routes.gif)

In [ ]:
!pip3 install boto3
!pip3 install leafmap

In [1]:
import boto3
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# Load environment variables from .env file
load_dotenv()

# Create a client for Amazon Location service
amazon_location_client = boto3.client(
    "location",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name = os.getenv("AWS_REGION"),
)

## Calculate optimized truck route (returning to the starting point)

In [3]:
response = amazon_location_client.calculate_route(
    CalculatorName='AWS-Esri-Demo',
    #DepartNow=True|False,
    DeparturePosition = [-0.1417089267, 51.5787616672],
    #DepartureTime = datetime(2015, 1, 1),
    DestinationPosition = [-0.1417089267, 51.5787616672],
    DistanceUnit = 'Kilometers',
    IncludeLegGeometry = True,
    #Key='string',
    TravelMode = 'Truck',
    TruckModeOptions = {
        'AvoidFerries': True,
        'AvoidTolls': True,
        'Dimensions': {
            'Height': 2,
            'Length': 2.57,
            'Unit': 'Meters',
            'Width': 1.4
        }
    },
    WaypointPositions = [
        [-0.1254974319, 51.50852485], 
        [-0.08229997944, 51.51856002], 
        [-0.1259365099, 51.53253756], 
        [-0.1425202953, 51.49654998]
    ]
)

In [5]:
len(response['Legs'])

5

In [18]:
response['Legs'][0]

{'Distance': 9.774851916292302,
 'DurationSeconds': 1499.63905556,
 'EndPosition': [-0.12549702207699787, 51.50852507962595],
 'Geometry': {'LineString': [[-0.1416738714698656, 51.578288235792705],
   [-0.14167388788668334, 51.5782882366452],
   [-0.14013997658194316, 51.57831998847711],
   [-0.13961998629387262, 51.577749992656486],
   [-0.1396199808852241, 51.577749998300746],
   [-0.14080996491173126, 51.577449992438005],
   [-0.14137997845270517, 51.57732999970002],
   [-0.14221997565415556, 51.57716999337086],
   [-0.1438099755035642, 51.57682999793471],
   [-0.14429998587995782, 51.57669998979005],
   [-0.14451996456035138, 51.57663000203214],
   [-0.14462996590942906, 51.57658998844097],
   [-0.14462997747905157, 51.57659000609797],
   [-0.14434998471303864, 51.57648000722438],
   [-0.14378997516379144, 51.5762199967865],
   [-0.14339998352418618, 51.57597999269396],
   [-0.1431399730863018, 51.575729997437364],
   [-0.1426499737371685, 51.57521999174288],
   [-0.142389987316505

In [17]:
response['Summary']

{'DataSource': 'Esri',
 'Distance': 36.61856671957526,
 'DistanceUnit': 'Kilometers',
 'DurationSeconds': 6048.49542343,
 'RouteBBox': [-0.1596899864241659,
  51.49654999262017,
  -0.08207854963730257,
  51.57831998847711]}

# Calculate all routes

In [86]:
df = pd.read_csv("truck_data.csv")

df

,truck_id,origin,destinations,width,height,length
0,0,"[-0.1715966336, 51.5393382483]","[[-0.1726801947, 51.51721979], [-0.08811283239...",1.9,1.8,5.6
1,1,"[-0.1103858794, 51.5438524668]","[[-0.1425202953, 51.49654998], [-0.1259365099,...",2.0,1.7,4.4
2,2,"[-0.16613175, 51.5333272373]","[[-0.1425202953, 51.49654998], [-0.08229997944...",1.7,1.7,5.6
3,3,"[-0.1555098879, 51.4940700205]","[[-0.08597997021, 51.50396], [-0.1425202953, 5...",2.0,1.8,5.3
4,4,"[-0.1599898433, 51.5593001851]","[[-0.1726801947, 51.51721979], [-0.1342243716,...",1.6,1.9,5.3
5,5,"[-0.1187149308, 51.5757533932]","[[-0.08229997944, 51.51856002], [-0.1254974319...",2.0,1.8,5.2


In [87]:
from ast import literal_eval
routes = []
for index, fila in df.iterrows():
    routes.append(amazon_location_client.calculate_route(
        CalculatorName='AWS-Esri-Demo',
        DeparturePosition = literal_eval(df.at[index,'origin']),
        DestinationPosition = literal_eval(df.at[index,'origin']),
        DistanceUnit = 'Kilometers',
        IncludeLegGeometry = True,
        TravelMode = 'Truck',
        TruckModeOptions = {
            'AvoidFerries': True,
            'AvoidTolls': True,
            'Dimensions': {
                'Height': df.at[index,'height'],
                'Length': df.at[index,'length'],
                'Unit': 'Meters',
                'Width': df.at[index,'width']
            }
        },
        WaypointPositions = literal_eval(df.at[index,'destinations'])
    ))


In [88]:
routes

[{'ResponseMetadata': {'RequestId': '33a6c58c-07f7-4590-bf6e-3edf64b8eee2',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'content-type': 'application/json',
    'content-length': '52894',
    'connection': 'keep-alive',
    'date': 'Sat, 07 Oct 2023 16:08:46 GMT',
    'x-amzn-requestid': '33a6c58c-07f7-4590-bf6e-3edf64b8eee2',
    'access-control-allow-origin': '*',
    'x-amz-apigw-id': 'McFWUEXPoAMEoHg=',
    'access-control-expose-headers': 'x-amzn-errortype,x-amzn-requestid,x-amzn-errormessage,x-amzn-trace-id,x-amz-apigw-id,date',
    'x-amzn-trace-id': 'Root=1-6521828d-64397d1418da678869cef903',
    'x-cache': 'Miss from cloudfront',
    'via': '1.1 d2165ef35515312478de7a439eead44e.cloudfront.net (CloudFront)',
    'x-amz-cf-pop': 'MAD53-P2',
    'x-amz-cf-id': '6tOG16IEZIcY_mTbbn1Ct4R1ENTA0C5PQId3S1d5nQNh1xkBMAn_3Q=='},
   'RetryAttempts': 0},
  'Legs': [{'Distance': 3.905304745668792,
    'DurationSeconds': 562.3426474430001,
    'EndPosition': [-0.17267993758776137, 51.51721992

# Display calculated routes on a Map

In [ ]:
import geojson
# Create a GeoJSON FeatureCollection
line_strings = []

def convert_to_geojson(line_strings):
    # Create a GeoJSON FeatureCollection
    features = []

    # Add LineString features
    for line_string_coords in line_strings:
        line_string = geojson.LineString(coordinates=line_string_coords)
        features.append(geojson.Feature(geometry=line_string))

    # Create the FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Serialize the GeoJSON object to a string
    geojson_str = geojson.dumps(feature_collection, sort_keys=True)

    return geojson_str

for i, route in enumerate(routes):
    line_strings = []
    for leg in route['Legs']:
        line_strings.append(leg['Geometry']['LineString'])

    # Call the function to convert to GeoJSON
    result_geojson = convert_to_geojson(line_strings)

    file_path = "data/truck/route_truck%d.geojson" % (i)

    # Write the GeoJSON string to the file
    with open(file_path, "w") as geojson_file:
        geojson_file.write(result_geojson)

    print(f"GeoJSON saved to {file_path}")

In [102]:
import leafmap

m = leafmap.Map(center=[51.50852507962595, -0.12549702207699787], zoom=12, draw_control=False, measure_control=False)
m.add_basemap("Esri.WorldTopoMap")
#m.add_xy_data('uk_stores_with_coordinates.csv', x="longitude", y="latitude", layer_name="All stores")
for i in range(6):
    file_path = "truck%d_origin.geojson" % (i)
    m.add_geojson("data/truck/%s" % (file_path), layer_name=file_path)
    file_path = "truck%d_destinations.geojson" % (i)
    m.add_geojson("data/truck/%s" % (file_path), layer_name=file_path)
    file_path = "route_truck%d.geojson" % (i)
    m.add_geojson("data/truck/%s" % (file_path), layer_name="truck%d_route" % (i))

m

Map(center=[51.50852507962595, -0.12549702207699787], controls=(ZoomControl(options=['position', 'zoom_in_text…